In [19]:
import os
import pandas as pd
import zipfile
import re
import shutil
import win32com.client as win32
from datetime import datetime,timedelta

In [20]:
brand = "雪肌精"
original_file_path = "D:/推广/"+brand+"/日报/每日数据源"
zipped_file_path = "D:/推广/"+brand+"/日报/每日数据源解压"
cleaned_file_path = "D:/推广/"+brand+"/日报"
excel_output_path = "D:/推广/"+brand+"/日报/日报"
zipfile_list = os.listdir(original_file_path)

In [21]:
def extract_zipfile(zipfile_path, target_folder):
    with zipfile.ZipFile(zipfile_path, 'r') as zip_ref:
        zip_ref.extractall(target_folder)

In [22]:
def csv_to_excel(csvfile_path,excelfile_path,name_of_sheet):
    try:
        df = pd.read_csv(csvfile_path,encoding="GBK")
    except:
        df = pd.read_csv(csvfile_path,encoding="UTF-8")
    df.to_excel(excelfile_path,index=False,sheet_name=name_of_sheet)

In [23]:
def clear_folder(folder_path):
    """清空文件夹（删除后重建）"""
    try:
        # 删除整个文件夹
        shutil.rmtree(folder_path)
        # 重新创建空文件夹
        os.makedirs(folder_path, exist_ok=True)
    except Exception as e:
        print(f"清空文件夹失败: {e}")

In [29]:
for file_need_zip in zipfile_list:
    extract_zipfile(original_file_path+"/"+file_need_zip,zipped_file_path)
    temp_csv_file = os.listdir(zipped_file_path)[0]
    excel_file_name = re.findall("\D+",file_need_zip)[0].replace(".zip","")
    csv_to_excel(zipped_file_path+"/"+temp_csv_file,cleaned_file_path+"/"+"/日报数据源/"+excel_file_name+".xlsx",name_of_sheet=excel_file_name)
    clear_folder(zipped_file_path)
    

<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Vincent\AppData\Local\Temp\ipykernel_21036\2045114461.py:4: SyntaxWarning: invalid escape sequence '\D'
  excel_file_name = re.findall("\D+",file_need_zip)[0].replace(".zip","")


In [25]:
def refresh_and_save_excel(input_path, output_path):
    """
    打开 Excel 文件，刷新所有数据（公式、透视表等），并另存为新文件
    需要已安装 Microsoft Excel 应用程序
    
    参数:
        input_path (str): 输入 Excel 文件路径
        output_path (str): 输出 Excel 文件路径
    """
    try:
        # 启动 Excel 应用程序
        excel = win32.gencache.EnsureDispatch('Excel.Application')
        excel.Visible = False  # 后台运行
        excel.DisplayAlerts = False # 强制覆盖不弹窗
        
        # 打开工作簿
        wb = excel.Workbooks.Open(os.path.abspath(input_path))
        
        # 刷新所有数据
        wb.RefreshAll()  # 刷新公式、透视表、数据连接等
        excel.CalculateUntilAsyncQueriesDone()  # 等待刷新完成
        
        # 另存为新文件（不覆盖原文件）
        wb.Save()
        wb.SaveAs(os.path.abspath(output_path))
        print(f"已刷新并另存为: {output_path}")
        
    except Exception as e:
        print(f"操作失败: {e}")
    finally:
        # 关闭工作簿和 Excel
        wb.Close(SaveChanges=False)
        excel.Quit()
        # 释放 COM 对象
        del excel


In [26]:
refresh_and_save_excel("D:/推广/"+brand+"/日报/雪肌精推广日报底表.xlsx",excel_output_path+"/"+brand+"推广日报"+datetime.strftime(datetime.today()-timedelta(days=1),"%m%d")+"_完整版.xlsx")

已刷新并另存为: D:/推广/雪肌精/日报/日报/雪肌精推广日报0722_完整版.xlsx
